# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'04-13-2021'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'04-13-2021'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2021-04-14 04:20:41,33.93911,67.709953,57364,2529,52013.0,2822.0,Afghanistan,147.357936,4.408688
1,NaN,NaN,NaN,Albania,2021-04-14 04:20:41,41.15330,20.168300,128752,2326,98903.0,27523.0,Albania,4473.973174,1.806574
2,NaN,NaN,NaN,Algeria,2021-04-14 04:20:41,28.03390,1.659600,118799,3137,82813.0,32849.0,Algeria,270.914879,2.640595
3,NaN,NaN,NaN,Andorra,2021-04-14 04:20:41,42.50630,1.521800,12614,121,11932.0,561.0,Andorra,16325.632563,0.959252
4,NaN,NaN,NaN,Angola,2021-04-14 04:20:41,-11.20270,17.873900,23697,554,22115.0,1028.0,Angola,72.101280,2.337849


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/4/21,4/5/21,4/6/21,4/7/21,4/8/21,4/9/21,4/10/21,4/11/21,4/12/21,4/13/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,56676,56717,56779,56873,56943,57019,57144,57160,57242,57364
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,126531,126795,126936,127192,127509,127795,128155,128393,128518,128752
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,117622,117739,117879,118004,118116,118251,118378,118516,118645,118799


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/4/21,4/5/21,4/6/21,4/7/21,4/8/21,4/9/21,4/10/21,4/11/21,4/12/21,4/13/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,2497,2508,2512,2512,2516,2521,2521,2521,2529,2529
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,2265,2274,2283,2291,2297,2304,2310,2317,2321,2326
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,3105,3108,3112,3116,3119,3123,3126,3130,3134,3137


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/4/21,4/5/21,4/6/21,4/7/21,4/8/21,4/9/21,4/10/21,4/11/21,4/12/21,4/13/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,51885,51902,51928,51940,51956,51961,51962,51989,52005,52013
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,93842,94431,95035,95600,96129,96672,97206,97723,98269,98903
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,81896,81994,82096,82192,82289,82392,82493,82600,82704,82813


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,4/4/21,4/5/21,4/6/21,4/7/21,4/8/21,4/9/21,4/10/21,4/11/21,4/12/21,4/13/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,6619,6620,6644,6675,6702,6710,6715,6723,6727,6734
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,20541,20542,20551,20573,20588,20600,20617,20631,20638,20652
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,2232,2232,2238,2239,2244,2245,2247,2247,2249,2252


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,4/4/21,4/5/21,4/6/21,4/7/21,4/8/21,4/9/21,4/10/21,4/11/21,4/12/21,4/13/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,102,102,102,102,103,103,103,103,103,103
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,301,301,301,301,300,300,300,300,300,300
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,55,55,55,55,55,55,55,55,55,55


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
652,1001.0,Autauga,Alabama,US,2021-04-14 04:20:41,32.539527,-86.644082,6734,103,NaN,NaN,"Autauga, Alabama, US",12053.195869,1.529552
689,1075.0,Lamar,Alabama,US,2021-04-14 04:20:41,33.779950,-88.096680,1393,34,NaN,NaN,"Lamar, Alabama, US",10090.546903,2.440775
690,1077.0,Lauderdale,Alabama,US,2021-04-14 04:20:41,34.901719,-87.656247,9223,227,NaN,NaN,"Lauderdale, Alabama, US",9946.187277,2.461238


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,31345312,563440,0.0
India,13873825,172085,12336036.0
Brazil,13599994,358425,11975121.0


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,31345312,563440,0.0,30781872.0,2021-04-14 04:20:41,37.950547,-91.419547
India,13873825,172085,12336036.0,1365704.0,2021-04-14 04:20:41,23.088275,81.806127
Brazil,13599994,358425,11975121.0,1266448.0,2021-04-14 04:20:41,-12.669522,-48.480493
France,5167265,99639,319880.0,4747746.0,2021-04-14 04:20:41,6.430808,-34.730285
Russia,4605444,101882,4233065.0,270497.0,2021-04-14 04:20:41,54.546312,62.120860


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,3705956,60541,0.0
Texas,2837160,49289,0.0
Florida,2134914,34120,0.0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,3705956,60541,0.0,3645415.0,2021-04-14 04:20:41,37.843962,-120.728594
Texas,2837160,49289,0.0,2787871.0,2021-04-14 04:20:41,31.660643,-98.653069
Florida,2134914,34120,0.0,2100794.0,2021-04-14 04:20:41,28.940755,-82.700744
New York,1967248,51258,0.0,1915990.0,2021-04-14 04:20:41,42.544151,-75.474183
Illinois,1285298,23826,0.0,1261472.0,2021-04-14 04:20:41,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,1226596,23505,0.0
Arizona,Maricopa,529271,9739,0.0
Illinois,Cook,515075,9776,0.0
Florida,Miami-Dade,463365,5993,0.0
Texas,Harris,383709,5990,0.0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,1226596,23505,0.0,1203091.0,2021-04-14 04:20:41,34.308284,-118.228241,6037.0
Arizona,Maricopa,529271,9739,0.0,519532.0,2021-04-14 04:20:41,33.348359,-112.491815,4013.0
Illinois,Cook,515075,9776,0.0,505299.0,2021-04-14 04:20:41,41.841448,-87.816588,17031.0
Florida,Miami-Dade,463365,5993,0.0,457372.0,2021-04-14 04:20:41,25.611236,-80.551706,12086.0
Texas,Harris,383709,5990,0.0,377719.0,2021-04-14 04:20:41,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-04-11,57160,128393,118516,12545,23457,1198,2532562,202817,29419,577007,...,4384610,144642,84922,3,174887,2693,268132,5357,90029,37288
2021-04-12,57242,128518,118645,12581,23549,1198,2551999,203327,29429,578950,...,4388296,147173,85114,3,175812,2705,270856,5446,90064,37307
2021-04-13,57364,128752,118799,12614,23697,1201,2579000,204053,29450,581263,...,4390801,149430,85291,3,176972,2714,272767,5507,90218,37330


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-04-11,2521,2317,3130,120,553,30,57779,3735,909,9687,...,127331,1462,634,0,1778,35,2860,1049,1226,1538
2021-04-12,2529,2321,3134,121,554,30,57957,3753,910,9706,...,127346,1533,634,0,1795,35,2883,1068,1227,1542
2021-04-13,2529,2326,3137,121,554,31,58174,3775,910,9748,...,127369,1595,634,0,1815,35,2901,1073,1229,1543


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-04-11,51989,97723,82600,11825,22058,938,2233140,181963,23085,535798,...,13494,110277,82760,1,157486,2429,232960,2053,88018,34873
2021-04-12,52005,98269,82704,11890,22093,938,2247124,182836,23087,538609,...,13608,113125,82943,1,158399,2445,234698,2059,88077,34901
2021-04-13,52013,98903,82813,11932,22115,947,2262875,183660,23104,541786,...,13660,116533,83089,1,159547,2445,237046,2070,88144,34932


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,4/4/21,4/5/21,4/6/21,4/7/21,4/8/21,4/9/21,4/10/21,4/11/21,4/12/21,4/13/21
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,6619,6620,6644,6675,6702,6710,6715,6723,6727,6734
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,1392,1392,1392,1392,1392,1392,1392,1392,1393,1393
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,9159,9162,9165,9178,9188,9196,9208,9215,9218,9223
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,2932,2932,2938,2947,2950,2950,2952,2953,2953,2953
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,15457,15459,15468,15468,15480,15491,15510,15524,15524,15541


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-04-11,518899,64782,0,849561,332153,3700471,478678,321586,98570,49,...,822085,2826977,390104,21202,2971,635552,376230,146169,645514,56873
2021-04-12,519071,65180,0,850236,332222,3704070,479590,324571,98925,49,...,824914,2831972,390289,21291,2998,636862,377952,146462,645964,56988
2021-04-13,520503,65336,0,850846,332446,3705956,481494,325689,99135,49,...,826371,2837160,390647,21388,3005,638910,379056,146790,647091,57070


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2021-04-11        6723   20631    2247  2562   6483    1213   2122   14284   
2021-04-12        6727   20638    2249  2564   6488    1213   2123   14286   
2021-04-13        6734   20652    2252  2564   6497    1213   2125   14301   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-04-11         3499     1820  ...    2653    611     3085      734   
2021-04-12         3500     1820  ...    2655    612     3090      742   
2021-04-13         3503     1821  ...    2658    612     3093      746   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-04-11           4085  3660  2142          0      897    640  
2021-04-12           4104  3665  2145          0      897    641  
2021-04-13           4107  3675  2153          0      898    642  

[3 rows x 3336 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-04-11,10712,313,0,17086,5661,60452,6157,7944,1578,0,...,12001,49212,2159,231,26,10472,5322,2745,7380,701
2021-04-12,10712,314,0,17086,5665,60495,6157,7957,1581,0,...,12015,49231,2159,233,26,10486,5329,2745,7383,701
2021-04-13,10722,314,0,17105,5673,60541,6168,7974,1587,0,...,12022,49289,2161,233,26,10506,5340,2747,7394,703


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2021-04-11         103     300      55   58    135      40     66     311   
2021-04-12         103     300      55   58    135      40     66     311   
2021-04-13         103     300      55   59    135      40     66     311   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2021-04-11          122       44  ...      30     11       31        7   
2021-04-12          122       44  ...      30     11       31        7   
2021-04-13          122       44  ...      30     11       31        7   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-04-11             37     9    12          0       26      5  
2021-04-12             37     9    12          0       26      5  
2021-04-13             37     9    12          0       26      5  

[3 rows x 3336 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-04-11,0.000280,0.001857,0.001166,0.003841,0.005401,0.000835,0.006063,0.003215,0.000476,0.003918,...,0.000395,0.023073,0.002337,0.0,0.006335,0.000371,0.008406,0.015353,0.001234,0.000402
2021-04-12,0.001435,0.000974,0.001088,0.002870,0.003922,0.000000,0.007675,0.002515,0.000340,0.003367,...,0.000841,0.017498,0.002261,0.0,0.005289,0.004456,0.010159,0.016614,0.000389,0.000510
2021-04-13,0.002131,0.001821,0.001298,0.002623,0.006285,0.002504,0.010580,0.003571,0.000714,0.003995,...,0.000571,0.015336,0.002080,0.0,0.006598,0.003327,0.007055,0.011201,0.001710,0.000617


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-04-11,0.000000,0.003030,0.001280,0.000000,0.005455,0.000000,0.002290,0.004032,0.0000,0.002587,...,0.000055,0.033946,0.0,NaN,0.010802,0.0,0.007752,0.017459,0.000000,0.000000
2021-04-12,0.003173,0.001726,0.001278,0.008333,0.001808,0.000000,0.003081,0.004819,0.0011,0.001961,...,0.000118,0.048564,0.0,NaN,0.009561,0.0,0.008042,0.018112,0.000816,0.002601
2021-04-13,0.000000,0.002154,0.000957,0.000000,0.000000,0.033333,0.003744,0.005862,0.0000,0.004327,...,0.000181,0.040444,0.0,NaN,0.011142,0.0,0.006243,0.004682,0.001630,0.000649


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-04-11,0.000520,0.005319,0.001297,0.004673,0.007675,0.002137,0.006924,0.004322,0.000087,0.006120,...,0.000000,0.055949,0.002107,0.0,0.006480,0.000000,0.007229,0.012827,0.013880,0.000316
2021-04-12,0.000308,0.005587,0.001259,0.005497,0.001587,0.000000,0.006262,0.004798,0.000087,0.005246,...,0.008448,0.025826,0.002211,0.0,0.005797,0.006587,0.007461,0.002923,0.000670,0.000803
2021-04-13,0.000154,0.006452,0.001318,0.003532,0.000996,0.009595,0.007009,0.004507,0.000736,0.005899,...,0.003821,0.030126,0.001760,0.0,0.007248,0.000000,0.010004,0.005342,0.000761,0.000888


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-04-11,0.000600,0.000000,NaN,0.000636,0.000301,0.000944,0.002587,0.000000,0.003737,0.0,...,0.000000,0.000505,0.000883,0.011305,0.000000,0.001934,0.000000,0.002847,0.000951,0.000000
2021-04-12,0.000331,0.006144,NaN,0.000795,0.000208,0.000973,0.001905,0.009282,0.003602,0.0,...,0.003441,0.001767,0.000474,0.004198,0.009088,0.002061,0.004577,0.002005,0.000697,0.002022
2021-04-13,0.002759,0.002393,NaN,0.000717,0.000674,0.000509,0.003970,0.003445,0.002123,0.0,...,0.001766,0.001832,0.000917,0.004556,0.002335,0.003216,0.002921,0.002239,0.001745,0.001439


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                  \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount Bullock   
2021-04-11      0.001191  0.000679  0.000000  0.000000  0.000463     0.0   
2021-04-12      0.000595  0.000339  0.000890  0.000781  0.000771     0.0   
2021-04-13      0.001041  0.000678  0.001334  0.000000  0.001387     0.0   

Province_State                                         ...   Wyoming  \
Admin2            Butler  Calhoun  Chambers  Cherokee  ...      Park   
2021-04-11      0.000000  0.00049  0.000000  0.000000  ...  0.000000   
2021-04-12      0.000471  0.00014  0.000286  0.000000  ...  0.000754   
2021-04-13      0.000942  0.00105  0.000857  0.000549  ...  0.001130   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-04-11      0.000000  0.000000  0.000000   0.000000  0.000000  0.000000   
2021-04-12      0.001637  0.001621  0.010899   0.004651  0.001366  0.001401   
2021-04-13      0.000000  0.000971  0.005391   0.000731  0.002729  0.003730   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2021-04-11            NaN  0.000000  0.000000  
2021-04-12            NaN  0.000000  0.001562  
2021-04-13            NaN  0.001115  0.001560  

[3 rows x 3336 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-04-11,0.000000,0.000000,NaN,-0.000351,0.000000,0.001126,0.000325,0.000000,0.000634,NaN,...,0.000000,0.000488,0.000000,0.000000,0.0,0.001339,0.000000,0.001094,0.000136,0.000000
2021-04-12,0.000000,0.003195,NaN,0.000000,0.000707,0.000711,0.000000,0.001636,0.001901,NaN,...,0.001167,0.000386,0.000000,0.008658,0.0,0.001337,0.001315,0.000000,0.000407,0.000000
2021-04-13,0.000934,0.000000,NaN,0.001112,0.001412,0.000760,0.001787,0.002136,0.003795,NaN,...,0.000583,0.001178,0.000926,0.000000,0.0,0.001907,0.002064,0.000729,0.001490,0.002853


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                  \
Admin2         Autauga Baldwin Barbour      Bibb Blount Bullock Butler   
2021-04-11         0.0     0.0     0.0  0.000000    0.0     0.0    0.0   
2021-04-12         0.0     0.0     0.0  0.000000    0.0     0.0    0.0   
2021-04-13         0.0     0.0     0.0  0.017241    0.0     0.0    0.0   

Province_State                            ... Wyoming                  \
Admin2         Calhoun Chambers Cherokee  ...    Park Platte Sheridan   
2021-04-11         0.0      0.0      0.0  ...     0.0    0.0      0.0   
2021-04-12         0.0      0.0      0.0  ...     0.0    0.0      0.0   
2021-04-13         0.0      0.0      0.0  ...     0.0    0.0      0.0   

Province_State                                                             
Admin2         Sublette Sweetwater Teton Uinta Unassigned Washakie Weston  
2021-04-11          0.0        0.0   0.0   0.0        NaN      0.0    0.0  
2021-04-12          0.0        0.0   0.0   0.0        NaN      0.0    0.0  
2021-04-13          0.0        0.0   0.0   0.0        NaN      0.0    0.0  

[3 rows x 3336 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-04-11,0.001015,0.002187,0.001120,0.003664,0.004990,0.004128,0.007297,0.004167,0.000369,0.004546,...,0.000299,0.028316,0.002328,5.820766e-11,0.006808,0.002168,0.008091,0.014287,0.001543,0.001518
2021-04-12,0.001225,0.001580,0.001104,0.003267,0.004456,0.002064,0.007486,0.003341,0.000354,0.003957,...,0.000570,0.022907,0.002294,2.910383e-11,0.006048,0.003312,0.009125,0.015450,0.000966,0.001014
2021-04-13,0.001678,0.001701,0.001201,0.002945,0.005370,0.002284,0.009033,0.003456,0.000534,0.003976,...,0.000570,0.019121,0.002187,1.455192e-11,0.006323,0.003320,0.008090,0.013326,0.001338,0.000815


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-04-11,0.000414,0.002942,0.001176,0.001085,0.003564,0.004589,0.003462,0.005117,1.357484e-35,0.003238,...,0.000208,0.039396,0.000181,NaN,0.010716,1.180129e-68,0.008624,0.015171,0.000213,0.000839
2021-04-12,0.001794,0.002334,0.001227,0.004709,0.002686,0.002295,0.003271,0.004968,5.500550e-04,0.002600,...,0.000163,0.043980,0.000090,NaN,0.010139,5.900647e-69,0.008333,0.016642,0.000514,0.001720
2021-04-13,0.000897,0.002244,0.001092,0.002355,0.001343,0.017814,0.003508,0.005415,2.750275e-04,0.003464,...,0.000172,0.042212,0.000045,NaN,0.010640,2.950323e-69,0.007288,0.010662,0.001072,0.001184


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2021-04-11,0.000325,0.005482,0.001259,0.004231,0.007887,0.008529,0.006611,0.004885,0.000182,0.006232,...,0.001221,0.035344,0.002101,0.0,0.005950,0.000250,0.007124,0.017851,0.010872,0.000498
2021-04-12,0.000317,0.005535,0.001259,0.004864,0.004737,0.004265,0.006437,0.004842,0.000135,0.005739,...,0.004835,0.030585,0.002156,0.0,0.005874,0.003419,0.007292,0.010387,0.005771,0.000650
2021-04-13,0.000235,0.005993,0.001288,0.004198,0.002866,0.006930,0.006723,0.004674,0.000435,0.005819,...,0.004328,0.030355,0.001958,0.0,0.006561,0.001709,0.008648,0.007865,0.003266,0.000769


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-04-11,0.000636,0.000905,NaN,0.000850,0.000419,0.000908,0.003024,0.001136,0.003820,6.749845e-119,...,0.000392,0.000819,0.000996,0.009349,0.000654,0.002279,0.001865,0.002959,0.001272,0.000352
2021-04-12,0.000484,0.003524,NaN,0.000822,0.000313,0.000940,0.002465,0.005209,0.003711,3.374923e-119,...,0.001917,0.001293,0.000735,0.006773,0.004871,0.002170,0.003221,0.002482,0.000984,0.001187
2021-04-13,0.001621,0.002959,NaN,0.000770,0.000494,0.000725,0.003217,0.004327,0.002917,1.687461e-119,...,0.001842,0.001562,0.000826,0.005665,0.003603,0.002693,0.003071,0.002361,0.001365,0.001313


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                  \
Admin2           Autauga   Baldwin   Barbour     Bibb    Blount  Bullock   
2021-04-11      0.001395  0.000709  0.000479  0.00034  0.000555  0.00016   
2021-04-12      0.000995  0.000524  0.000684  0.00056  0.000663  0.00008   
2021-04-13      0.001018  0.000601  0.001009  0.00028  0.001025  0.00004   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2021-04-11      0.000597  0.000493  0.000322  0.000398  ...  0.000269   
2021-04-12      0.000534  0.000317  0.000304  0.000199  ...  0.000511   
2021-04-13      0.000738  0.000683  0.000580  0.000374  ...  0.000821   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2021-04-11      0.000002  0.000035  0.001376   0.000484  0.000476  0.000183   
2021-04-12      0.000819  0.000828  0.006137   0.002568  0.000921  0.000792   
2021-04-13      0.000410  0.000899  0.005764   0.001649  0.001825  0.002261   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2021-04-11           -1.0  0.000639  0.000406  
2021-04-12           -1.0  0.000319  0.000984  
2021-04-13           -1.0  0.000717  0.001272  

[3 rows x 3336 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2021-04-11,0.000904,1.872082e-08,NaN,0.000490,0.000064,0.001537,0.000579,0.000156,0.001408,NaN,...,0.000235,0.001085,0.001057,0.000695,3.051758e-07,0.001290,0.000357,0.001647,0.000459,0.000024
2021-04-12,0.000452,1.597453e-03,NaN,0.000245,0.000385,0.001124,0.000290,0.000896,0.001655,NaN,...,0.000701,0.000736,0.000528,0.004677,1.525879e-07,0.001313,0.000836,0.000823,0.000433,0.000012
2021-04-13,0.000693,7.987267e-04,NaN,0.000678,0.000899,0.000942,0.001038,0.001516,0.002725,NaN,...,0.000642,0.000957,0.000727,0.002338,7.629395e-08,0.001610,0.001450,0.000776,0.000961,0.001433


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                              \
Admin2           Autauga   Baldwin   Barbour          Bibb    Blount   
2021-04-11      0.000672 -0.000208  0.000005 -1.209598e-13  0.002580   
2021-04-12      0.000336 -0.000104  0.000002 -6.047992e-14  0.001290   
2021-04-13      0.000168 -0.000052  0.000001  8.620690e-03  0.000645   

Province_State                                                        ...  \
Admin2           Bullock        Butler   Calhoun  Chambers  Cherokee  ...   
2021-04-11      0.000006  9.300395e-16  0.000420  0.000682  0.005814  ...   
2021-04-12      0.000003  4.650198e-16  0.000210  0.000341  0.002907  ...   
2021-04-13      0.000002  2.325099e-16  0.000105  0.000170  0.001454  ...   

Province_State       Wyoming                                            \
Admin2                  Park        Platte      Sheridan      Sublette   
2021-04-11      3.288965e-08  1.334978e-24  2.484032e-10  1.091258e-30   
2021-04-12      1.644483e-08  6.674892e-25  1.242016e-10  5.456288e-31   
2021-04-13      8.222413e-09  3.337446e-25  6.210080e-11  2.728144e-31   

Province_State                                                    \
Admin2         Sweetwater         Teton         Uinta Unassigned   
2021-04-11       0.000007  3.483500e-18  1.770293e-25  -0.501961   
2021-04-12       0.000003  1.741750e-18  8.851466e-26  -0.501961   
2021-04-13       0.000002  8.708749e-19  4.425733e-26  -0.501961   

Province_State                              
Admin2              Washakie        Weston  
2021-04-11      6.780167e-23  1.439956e-21  
2021-04-12      3.390084e-23  7.199780e-22  
2021-04-13      1.695042e-23  3.599890e-22  

[3 rows x 3336 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,...,NaN,0,0,96d23f888c995b9a7f3b4b864de6414f45c728ff,0,0,0,0,0,NaN
4,20210307,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20210307,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
1,20210307,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2,20210307,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
3,20210307,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,NaN,0,0,96d23f888c995b9a7f3b4b864de6414f45c728ff,0,0,0,0,0,NaN
4,20210307,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886,NaN,NaN,NaN,totalTestsViral,1731628,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818,69092.0,2480716.0,NaN,totalTestsViral,2736442,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0,NaN,2140.0,NaN,totalTestsViral,2140,NaN,NaN,NaN,...,NaN,0,0,96d23f888c995b9a7f3b4b864de6414f45c728ff,0,0,0,0,0,NaN
2021-03-07,AZ,826454,56519.0,3073010.0,NaN,totalTestsViral,7908105,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,dataQualityGrade,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2021-03-07,AK,56886.0,NaN,NaN,NaN,totalTestsViral,1731628.0,33.0,1293.0,NaN,...,NaN,0,0,dc4bccd4bb885349d7e94d6fed058e285d4be164,0,0,0,0,0,NaN
2021-03-07,AL,499819.0,107742.0,1931711.0,NaN,totalTestsPeopleViral,2323788.0,494.0,45976.0,NaN,...,NaN,-1,0,997207b430824ea40b8eb8506c19a93e07bc972e,0,0,0,0,0,NaN
2021-03-07,AR,324818.0,69092.0,2480716.0,NaN,totalTestsViral,2736442.0,335.0,14926.0,141.0,...,NaN,22,11,50921aeefba3e30d31623aa495b47fb2ecc72fae,0,0,0,0,0,NaN
2021-03-07,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,NaN,...,NaN,0,0,96d23f888c995b9a7f3b4b864de6414f45c728ff,0,0,0,0,0,NaN
2021-03-07,AZ,826454.0,56519.0,3073010.0,NaN,totalTestsViral,7908105.0,963.0,57907.0,273.0,...,NaN,5,44,0437a7a96f4471666f775e63e86923eb5cbd8cdf,0,0,0,0,0,NaN
